In [ ]:
# import libraries
import pandas as pd
import numpy as np
from scipy.stats import norm

# load data
participant_df = pd.read_csv("/workspaces/enterntainment720/Nutrition/input/DII_Input_20240602.csv")
reference_df = pd.read_csv("/workspaces/enterntainment720/Nutrition/data/raw/DII_Reference_Table.csv")

# Filter reference table to nutrients present in participant data
nutrients_available = [n for n in reference_df["nutrient"] if n in participant_df.columns]
reference_df = reference_df[reference_df["nutrient"].isin(nutrients_available)]

# compute DII
def compute_dii(row):
    score = 0
    for _, r in reference_df.iterrows():
        nutrient = r["nutrient"]
        value = row.get(nutrient)
        if pd.isna(value):
            continue
        z = (value - r["global_mean"]) / r["global_sd"]
        p = 2 * norm.cdf(z) - 1
        dii = p * r["weight"]
        score += dii
    return score

participant_df["DII_score"] = participant_df.apply(compute_dii, axis=1)

# Output the result
print(participant_df[["SEQN", "DII_score"]].head())
participant_df[["SEQN", "DII_score"]].to_csv("/workspaces/enterntainment720/Nutrition/data/output/dii_scores_python_20240602.csv", index=False)


    SEQN  DII_score
0      1   0.000000
1      2  -7.004394
2      3   7.004394
3  62161  -0.050112
4  62164  -0.480223


In [3]:
# Load data
participant_df = pd.read_csv("/workspaces/enterntainment720/Nutrition/input/DII_Input_20240602.csv")
reference_df = pd.read_csv("/workspaces/enterntainment720/Nutrition/data/raw/DII_Reference_Table.csv")

# Filter reference table to nutrients present in participant data
nutrients_available = [n for n in reference_df["nutrient"] if n in participant_df.columns]
reference_df = reference_df[reference_df["nutrient"].isin(nutrients_available)]

# Compute Z-scores for each nutrient
zscore_df = participant_df[["SEQN"]].copy()
for _, row in reference_df.iterrows():
    nutrient = row["nutrient"]
    mean = row["global_mean"]
    sd = row["global_sd"]
    zscore_df[nutrient + "_z"] = (participant_df[nutrient] - mean) / sd

# Export Z-scores
zscore_df.to_csv("/workspaces/enterntainment720/Nutrition/data/output/z_scores_by_nutrient_20240606.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Load data
participant_df = pd.read_csv("/workspaces/enterntainment720/Nutrition/input/DII_Input_20240602.csv")
reference_df = pd.read_csv("/workspaces/enterntainment720/Nutrition/data/raw/DII_Reference_Table.csv")

# Filter reference table to nutrients present in participant data
nutrients_available = [n for n in reference_df["nutrient"] if n in participant_df.columns]
reference_df = reference_df[reference_df["nutrient"].isin(nutrients_available)]

# Initialize columns
detailed_dii = participant_df[["SEQN"]].copy()
total_scores = []

# Compute values
for index, row in participant_df.iterrows():
    total_score = 0
    row_results = {}

    for _, r in reference_df.iterrows():
        nutrient = r["nutrient"]
        value = row.get(nutrient)
        if pd.isna(value):
            z = np.nan
            p = np.nan
            contrib = np.nan
        else:
            z = (value - r["global_mean"]) / r["global_sd"]
            p = 2 * norm.cdf(z) - 1
            contrib = p * r["weight"]
            total_score += contrib

        row_results[f"{nutrient}"] = value
        row_results[f"{nutrient}_z"] = z
        row_results[f"{nutrient}_c"] = p
        row_results[f"{nutrient}_dii"] = contrib

    total_scores.append(total_score)
    for col, val in row_results.items():
        detailed_dii.at[index, col] = val

detailed_dii["score_dii"] = total_scores

# Save full detailed output
detailed_dii.to_csv("/workspaces/enterntainment720/Nutrition/data/output/detailed_dii_table_20240606.csv", index=False)

/tmp/ipykernel_497/1746897332.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  detailed_dii.at[index, col] = val
/tmp/ipykernel_497/1746897332.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  detailed_dii.at[index, col] = val
/tmp/ipykernel_497/1746897332.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = 